<a href="https://colab.research.google.com/github/codyub/ESAA/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD_%EA%B0%80%EC%9D%B4%EB%93%9C_ch8_pg_487~496.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04 텍스트 분류 실습 - 20 뉴스 그룹 분류


- 사이킷런의 예제 데이터
- 텍스트 피처 벡터화로 변환하면 일반적으로 희소 행렬 형태가 됨
- 희소 행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘은 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등임
- 로지스틱 회귀를 이용해 분류 수행
- 텍스트 기반의 분류는 먼저, 텍스트를 정규화한 뒤 피처 벡터화를 적용, 이후에 적합한 머신러닝 알고리즘을 적용해 분류를 학습/예측/평가

# 텍스트 정규화


In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data=fetch_20newsgroups(subset='all', random_state=156)

- 어떤 키 값을 갖고 있는지 확인해보자

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


-filenames: API가 인터넷에서 내려받아 로컬 컴퓨터에 저장하는 디렉터리와 파일명을 지칭함

- target class 구성 확인

In [3]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


- target 클래스의 값: 0~19까지 20개로 구성

- 개별 데이터가 텍스트로 어떻게 구성되어 있는지 데이터를 한 개만 추출해 값을 확인

In [4]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

- 텍스트 데이터에는 뉴스 그룹 기사의 내용만이 아닌, 뉴스그룹 제목, 작성자, 소속, 이메일 등 다양한 정보를 갖고 있음
- 이 중 내용을 제외하고 제목 등의 다른 정보는 제거 -> target 클래스의 값과 유사한 데이터를 갖고 있는 경우가 많기 때문
- 이 피처들을 포함하게 되면 왠만한 ML 알고리즘을 적용해도 상당히 높은 예측 성능을 보이므로, 이들 헤더와 푸터 정보를 포함하는 것은 이 장에서 수행하려는 텍스트 분석의 의도를 벗어남
- remove: 뉴스그룹 기사와 헤더, 푸터 등을 제거 가능
- subset: 학습 데이터와 테스트 데이터를 분리해 내려받음


In [5]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
train_news=fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)
X_train=train_news.data
y_train=train_news.target

# subset='test'로 테스트용 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
test_news=fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), random_state=156)
X_test=test_news.data
y_test=test_news.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))
     

학습 데이터 크기 11314, 테스트 데이터 크기 7532


# 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
- 학습 데이터는 11314개의 뉴스그룹 문서가 리스트 형태로 주어지고, 테스트 데이터는 7532개의 문서가 역시 리스트 형태로 주어짐 
- CountVectorizer를 이용해 학습 데이터와 테스트 데이터의 텍스트를 피처 벡터화
- 유의점: 테스트 데이터에서 CountVectorizer를 적용할 때는 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 데이터를 변환해야 함 --> 그래야 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터를 CountVectorizer로 변환할 피처 개수가 같아짐

- 테스트 데이터의 피처 벡터화 시 fit_transform()을 사용하면 안됨 
- CountVectorizer.fit_transform(테스트 데이터)를 테스트 데이터 세트에 적용하면 테스트 데이터 기반으로 다시 수행하고 변환해서 학습 시 사용된 피처 개수와 예측 시 사용할 피처 개수가 달라짐

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행.
cnt_vect=CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect=cnt_vect.transform(X_train)

# 학습 데이터로 fit된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect=cnt_vect.transform(X_test)

print('학습 데이터 테스트의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습 데이터 테스트의 CountVectorizer Shape: (11314, 101631)


- 학습 데이터를 CountVectorizer로 피처를 추출한 결과 11314개 문서에서 피처(단어)가 101631개로 만들어짐

- 이렇게 피처 벡터화된 데이터에 로지스틱 회귀를 적용해 뉴스그룹에 대한 분류를 예측

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf=LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred=lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.606


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


- Count 기반으로 피처 벡터화가 적용된 데이터 세트에 대한 로지스틱 회귀의 예측 정확도는 약 0.606





- Count 기반 말고 TF-IDF 기반으로 벡터화를 변경해 수행

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화로 학습 데이터 세트와 테스트 데이터 세트 변환.
tfidf_vect=TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect=tfidf_vect.transform(X_train)
X_test_tfidf_vect=tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf=LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred=lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


- TF-IDF가 단순 카운트보다는 높은 정확도 제공. 
- 일반적으로 문서 내에 텍스트가 많고 많은 문서를 가지는 텍스트 분석에서 카운트 벡터화보다는 TF-IDF가 좋은 예측 결과를 도출

- 텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지 방법은 최적의 ML 알고리즘을 선택하는 것과 최상의 피처 전처리를 수행하는 것. 
- 텍스트 정규화나 count/tf-idf 기반 피처 벡터화를 어떻게 효과적으로 적용했는지가 텍스트 기반의 머신러닝 성능에 큰 영향을 미칠 수 있음. 


- 앞의 TF-IDF 벡터화는 기본 파라미터만 적용했지만, 좀 더 다양한 파라미터를 적용하기
- 텍스트 분석에서 머신러닝 모델의 성능을 향상시키는 중요한 2가지방법
1) 최적의 ML 알고리즘을 선택
2) 최상의 피처 전처리를 수행하는 것

In [9]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용.
tfidf_vect=TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect=tfidf_vect.transform(X_train)
X_test_tfidf_vect=tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf=LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred=lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.692


- 이번에는 GridSearchCV를 이용해 로지스틱 회귀의 하이퍼 파라미터 최적화를 수행. 
- 로지스틱 회귀의 C 파라미터만 변경하며 최적의 C값을 찾고 이 C값으로 학습된 모델에서 테스트 데이터로 예측해 성능을 평가.



In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 폴드 세트로 설정.
params={'C':[5, 10]} #시간 문제로 0.01, 0.1, 1, 5, 10 중 5, 10만 모델링링
grid_cv_lr=GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter:', grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가
pred=grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 2 candidates, totalling 6 fits


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che


-  C가 10일 때 가장 좋은 예측 성능을 나타냄



# 사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합
- 사이킷런의 파이프라인 클래스 이용하면 피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성을 한 번에 진행 가능
- 일반적인 머신러닝에서 파이프라인이란 데이터의 가공, 변환 등의 전처리와 알고리즘 적용을 한꺼번에 스트림 기반으로 처리한다는 의미. -> 더 직관적인 코드 작성 가능. 또한 수행 시간 절약 가능.
- 일반적으로 사이킷런 파이프라인은 텍스트 기반의 피처 벡터화뿐만 아니라 모든 데이터 전처리 작업과 Estimator를 결합 가능: 예를 들어 스케일링 또는 벡터 정규화, PCA 등의 변환 작업과 분류, 회귀 등의 Estimator를 한 번에 결합하는 것.

- 위의 텍스트 분류 예제 코드를 Pipeline을 이용해 다시 작성해보면 다음과 같이 선언.

In [ ]:
pipeline=Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english')),('lr_clf', LogisticRegression(random_state=156))])

- TfidfVectorizer 객체를 tfidf_vect 라는 객체 변수 명으로, LogisticRegression 객체를 lr_clf 라는 객체 변수 명으로 생성한 뒤 이 두 개의 객체를 파이프라인으로 연결하는 파이프라인을 생성한다는 의미.

- 다음 코드를 보면 기존 TfidfVectorizer의 학습 데이터와 테스트 데이터에 대한 fit()과 transform() 수행을 통한 피처 벡터화와 LogisticRegressor의 fit과 predict 수행을 통한 머신러닝 모델의 학습과 예측이 파이프라인의 fit과 predict로 통일돼 수행됨을 알 수 있음
-> 머신러닝 코드 더 직관적이고 쉽게 작성

In [ ]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 pipeline 생성
pipeline=Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
                   ('lr_clf', LogisticRegression(C=10))])

# 별도의 TfidfVectorizer 객체의 fit, transform과 LogisticRegression의 fit, predict가 필요없음
# Pipeline의 fit()과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred=pipeline.predict(X_test)
print('Pipeline을 통한 LogisticRegression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))
     

- 사이킷런은 gridsearchcv 클래스의 생성 파라미터로 pipeline을 입력해 pipeline 기반에서도 하이퍼 파라미터 튜닝을 gridsearchcv 방식으로 진행할 수 있게 지원

- 다음 예제는 GridSearchCV에 파이프라인을 입력하면서 tfidfVectorizer의 파라미터와 logisticregression의 하이퍼 파라미터를 함께 최적화

- 문제점은 pipeline+gridsearchcv를 적용해서 모두의 파라미터를 최적화하려면 시간이 너무 오래 걸린다는 것.

In [ ]:
from sklearn.pipeline import Pipeline

pipeline=Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english')),
                   ('lr_clf', LogisticRegression())])

# Pipeline에 기술된 각각의 객체 변수에 언더바 2개를 붙여 GridSearchCV에 사용될 파라미터/하이퍼 파라미터 이름과 값을 설정
#이 때 param_grid의 입력 값 설정이 기존과 약간 다르다. 딕셔너리 키 값이 하이퍼 파라미터명이 객체 변수명과 결합돼 제공.
#피처 벡터화 객체의 파라미터와 estimator 객체의 하이퍼 파라미터를 각각 구별할 수 있어야 하는데, 이때 개별 객체 명과 파라미터명/하이퍼 파라미터명을 결합해 키 값으로 할당하는 것.
params={'tfidf_vect__ngram_range':[(1,1), (1,2), (1,3)],
        'tfidf_vect__max_df':[100, 300,700],
        'lr_clf__C':[1,5,10]}

# GridSearchCV 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe=GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred=grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

- 교재: max_df=700, ngram_ragne=(1,2), C=10일 때 가장 좋은 검증 세트 성능 수치가 도출되며, 하이퍼 파라미터 튜닝 결과를 적용해도 크게 개선되지는 않음
- 램 문제로 돌아가지 않음